In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from neurosynth import meta, decode, network, Dataset # to work with the neurosynth dataset
import nibabel as nib
import pickle
import pandas
import cortex
%matplotlib inline
class MacOSFile(object):

    def __init__(self, f):
        self.f = f

    def __getattr__(self, item):
        return getattr(self.f, item)

    def read(self, n):
        # print("reading total_bytes=%s" % n, flush=True)
        if n >= (1 << 31):
            buffer = bytearray(n)
            idx = 0
            while idx < n:
                batch_size = min(n - idx, 1 << 31 - 1)
                # print("reading bytes [%s,%s)..." % (idx, idx + batch_size), end="", flush=True)
                buffer[idx:idx + batch_size] = self.f.read(batch_size)
                # print("done.", flush=True)
                idx += batch_size
            return buffer
        return self.f.read(n)

    def write(self, buffer):
        n = len(buffer)
        print("writing total_bytes=%s..." % n, flush=True)
        idx = 0
        while idx < n:
            batch_size = min(n - idx, 1 << 31 - 1)
            print("writing bytes [%s, %s)... " % (idx, idx + batch_size), end="", flush=True)
            self.f.write(buffer[idx:idx + batch_size])
            print("done.", flush=True)
            idx += batch_size
def pickle_dump(obj, file_path):
    with open(file_path, "wb") as f:
        return pickle.dump(obj, MacOSFile(f), protocol=pickle.HIGHEST_PROTOCOL)
def pickle_load(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(MacOSFile(f))

# Visualization on freesurfer brain
Run the below cell if all you want to do is to locally run the contents of paulscotti.github.io/EduCortex

In [ ]:
subject = 'fsaverage'
anat = cortex.db.get_anat(subject, 'raw')
anat_data = anat.get_data()
anat_vol = cortex.Volume.empty(subject, 'atlas_2mm')

cortex.webgl.show(data=anat_vol, template='my_template.html', labels_visible=(""), overlays_visible=(""), recache=True)

Generating new ctm file...
wm
wm
inflated
inflated
Started server on port 26965


# Visualization of brain: Top 3 PCA of Neurosynth

In [2]:
comps = np.load('PCA/components.npz')['arr_0']
comps = comps.transpose(0,3,2,1)
comps.reshape(10,-1).shape

(10, 902629)

In [3]:
def make_colorvol(proj, meanstds, rgbpcs, mask, flips=tuple(), clip_lim=3):
    rgbdata = proj[rgbpcs,:]
    for f in flips:
        rgbdata[f] *= -1
    zrgb = (rgbdata.T / meanstds).T
    #zrgb = npp.rs(rgbdata.T).T
    crgb = np.clip(zrgb, -clip_lim, clip_lim) ## clip to 3 standard deviations
    srgb = crgb/clip_lim/2.0 + 0.5 ## scale to 0..1
    
    rgbvol = np.zeros([3]+list(mask.shape))
    rgbvol[0] = vox_to_mask(srgb[0], mask)
    rgbvol[1] = vox_to_mask(srgb[1], mask)
    rgbvol[2] = vox_to_mask(srgb[2], mask)

    colorvol = (rgbvol*255).astype(np.uint8)
    return colorvol.transpose(1,2,3,0)

def vox_to_mask(data, mask):
    dvol = mask.copy().astype(np.float)
    dvol[mask>0] = data
    return dvol

In [4]:
rescale = lambda x: x/x.std()

In [5]:
flat_comps = comps.reshape(10,-1)

colorvol = make_colorvol(flat_comps, flat_comps.std(1)[:3], [0,1,2], np.ones(comps[0].shape), clip_lim=4)

In [6]:
rgbvol = cortex.dataset.normalize((colorvol, 'fsaverage', 'atlas_2mm'))
rgbvol.alpha.vmin = 0

In [19]:
# cortex.webgl.show(data=rgbvol, template='my_template.html', labels_visible=(""), overlays_visible=(""), recache=True)

# to make your own localhost server:
cortex.webgl.make_static("static",data=rgbvol, template='my_template.html', labels_visible=(""), overlays_visible=(""), recache=True)

Generating new ctm file...
wm
wm
inflated
inflated
Started server on port 33808


KeyboardInterrupt: 

# Individual PCAs

In [68]:
concat_maps = pickle_load("PCA/concat_maps.p")

In [69]:
ncomp = 10
pca = PCA(copy=True, iterated_power='auto', n_components=ncomp, random_state=None,
  svd_solver='randomized', tol=0.0, whiten=False).fit(concat_maps)
print(pca.explained_variance_ratio_)  
print(pca.singular_values_)

[0.10724451 0.07186315 0.04695774 0.03718591 0.03011859 0.02566107
 0.01888798 0.01386358 0.01315753 0.01177422]
[7371.67572625 6034.36808339 4877.89306864 4340.78108748 3906.57327303
 3605.9198214  3093.65168398 2650.43018107 2582.05703612 2442.5570153 ]


In [70]:
# this is a list of all neurosynth terms
terms = pandas.read_csv('PCA/analysis_filter_list.tsv',delimiter='\t')
# but we're only interested in the terms that are non-anatomical
terms_anatfilter = pandas.read_csv('PCA/neurosynth_terms_anatfilter.txt',delimiter='\t')

# only keep terms that have a 'keep' value of 1 and n 'anatomical' value of 0
kept_terms_anatfilter = terms_anatfilter['term'][np.logical_and(terms_anatfilter['keep']==1, terms_anatfilter['anatomical']==0)]
# we will take these terms and find their location in a list that is ordered alphabetically
kept_terms = terms['term'][terms['keep']==1]
anatfilter_indices = [i for i,k in enumerate(list(kept_terms)) if k in list(kept_terms_anatfilter)]
kept_terms_anatfilter = kept_terms.iloc[anatfilter_indices]

concat_maps_anatfilter = concat_maps[anatfilter_indices]

In [71]:
components = np.array([dataset.masker.unmask(pca.components_[i],output='array') for i in range(ncomp)])
# np.savez('PCA/components',components)
# components = np.load("PCA/components.npz")

In [72]:
term_weights = pca.transform(concat_maps_anatfilter)
# term_weights = np.load("PCA/term_weights.npz")['arr_0']

In [112]:
for i in range(ncomp):
    a = term_weights[:,i].argsort()[::-1];
    print('\n',i);
    for j in range(5):
        print(kept_terms_anatfilter.iloc[a[j]]);
        print(term_weights[a[j],i]);
    print('\n')
    for k in range(5):
        print(kept_terms_anatfilter.iloc[a[-(k+1)]]);
        print(term_weights[a[-(k+1)],i]);


 0
motor
1317.6320905496177
movements
1082.6326032585494
movement
900.2029858277143
hand
887.2103314246036
finger
861.8239098595906


emotional
-608.1144302912776
negative
-514.7500530939691
reward
-500.44840115908005
social
-465.5854168413259
neutral
-448.30683321387113

 1
semantic
633.6148278762525
language
629.3179671473943
words
615.9370102636225
visual
603.6445620205632
word
595.8302894678343


pain
-605.0312709256932
motor
-578.6276868825615
reward
-496.64057976804315
finger
-444.4839686401221
movement
-434.66160568566374

 2
auditory
813.2986947960751
speech
679.48458442428
sounds
590.4998322822294
listening
517.4453536556493
acoustic
465.65334047620195


memory
-623.852572846215
task
-589.5702804802609
retrieval
-466.03949657232926
working memory
-452.0013096511467
working
-448.105861236009

 3
visual
383.39499998676183
motion
356.8537604490542
object
310.5796624466409
spatial
282.95892229130527
faces
277.5914845695568


auditory
-413.6016411663783
semantic
-404.1977385799328

## Anatomical labels

In [7]:
# load atlas
lh_aparc_file = os.path.join('atlases','lh.HCP-MMP1.annot')
rh_aparc_file = os.path.join('atlases','rh.HCP-MMP1.annot')
lpinds, lpstats, lpnames = nib.freesurfer.read_annot(lh_aparc_file)
lpinds_orig, lpstats, lpnames = nib.freesurfer.read_annot(lh_aparc_file, True)
lpinds[lpinds_orig==0] = -1
rpinds, rpstats, rpnames = nib.freesurfer.read_annot(rh_aparc_file)
rpinds_orig, rpstats, rpnames = nib.freesurfer.read_annot(rh_aparc_file, True)
rpinds[rpinds_orig==0] = -1

aparc = cortex.Vertex(np.hstack([lpinds, rpinds]),'fsaverage')